# **Лабораторная работа 1**
## Геометрические предикаты и расстояния (2D)

**Цель:** собрать базовый набор 2D-предикатов, который дальше используется во многих алгоритмах вычислительной геометрии.


### Оценивание (за полную лабораторную работу)
- **8:** всё реализовано корректно, есть тесты и визуализации, объяснены краевые случаи.
- **6–7:** мелкие недочёты в тестах/оформлении.
- **4–5:** часть функций реализована или есть ошибки на краях.
- **1–3:** попытка есть, но ключевые функции некорректны.
- **0:** не выполнено.

---

## Как работать в Colab
1. Откройте ноутбук в Google Colab.
2. Последовательно выполняйте ячейки сверху вниз.
3. В местах `# TODO` допишите код.
4. В конце запустите ячейку **Self-check** — она должна завершиться без ошибок.


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

EPS = 1e-9

## 1) **Предикаты и расстояния**


**Задача:** реализовать предикат `orient(A, B, C)`.

- Возвращает `+1`, если тройка точек образует **против часовой стрелки** (CCW).
- Возвращает `-1`, если **по часовой стрелке** (CW).
- Возвращает `0`, если точки **коллинеарны** (с учётом `EPS`).

In [ ]:
def orient(A,B,C):
    # TODO
    raise NotImplementedError

**Задача:** реализовать `dist_point_line(P, a, b, c)`.


*   Возвращает расстояние от точки `P(x,y)` до прямой `ax+by+c=0`



Краевые случаи
- Если `a = 0` и `b = 0`, это не прямая (уравнение некорректно).  
  В таком случае корректно **бросить исключение**: `raise ValueError(...)`.

In [ ]:
def dist_point_line(P, a,b,c):
    # TODO
    raise NotImplementedError

**Задача:** реализовать `on_segment`.

- Возвращает булево значение (`True` или `False`): принадлежность точки `P(x,y)` отрезку `[A,B]`.

- Необходимо проверить коллинеарность точки `P(x,y)` концам отрезка `[A,B]`.  
  В ином случае корректно **бросить исключение**: `raise ValueError(...)`.
  
  **Обратите внимание на крайние случаи:**
- `a = b` (вырожденный отрезок)
- `p` совпадает с концом отрезка

In [ ]:
def on_segment(A,B,P):
    # TODO
    raise NotImplementedError

**Задача**: реализовать `segments_intersect`.

- Возвращает булево значение (`True` или `False`): пересекаются ли отрезки `[A,B]` и `[C,D]`.

Учесть все случаи: пересечение, касание (в вершине/на стороне), коллинеарность и перекрытие, отсутствие пересечения.

In [ ]:
def segments_intersect(A,B,C,D):
    # TODO
    raise NotImplementedError

## 2) **Многоугольники**

**Задача**: реализовать `polygon_area`.

*  Возвращает число (float): ориентированную площадь простого многоугольника poly.



> `poly` — список вершин в порядке обхода, например, `square = [(0,0),(1,0),(1,1),(0,1)]`



In [ ]:
def polygon_area(poly):
    # Ориентированная площадь (shoelace)
    # TODO
    raise NotImplementedError

**Задача**: реализовать `point_in_polygon`.

*  Возвращает булево значение (`True` или `False`): принадлежит ли точка `Q(x,y)` многоугольнику `poly`.

> `poly` — список вершин в порядке обхода, например, `square = [(0,0),(1,0),(1,1),(0,1)]`

In [ ]:
def point_in_polygon(Q, poly):
    # Лучевой тест. Границу считаем внутри.
    # TODO
    raise NotImplementedError

### 3) **Self-check**


In [ ]:
# self-check
def _assert(cond, msg="failed"):
    if not cond:
        raise AssertionError(msg)

_assert(orient((0,0),(1,0),(0,1)) > 0)
_assert(orient((0,0),(1,0),(0,-1)) < 0)
_assert(abs(orient((0,0),(1,0),(2,0))) <= 1e-9)

d = dist_point_line((1,0), 1,-1,-0.5)
_assert(abs(d - (0.5/math.sqrt(2))) < 1e-9)

seg_tests = [
    ((0,0),(2,0),(1,-1),(1,1), True),
    ((0,0),(2,0),(3,-1),(3,1), False),
    ((0,0),(2,0),(2,0),(3,0), True),
    ((0,0),(3,0),(1,0),(2,0), True),
]
for A,B,C,D,ans in seg_tests:
    _assert(segments_intersect(A,B,C,D) == ans)

square = [(0,0),(1,0),(1,1),(0,1)]
A = polygon_area(square)
_assert(abs(A-1.0) < 1e-9)
_assert(point_in_polygon((0.5,0.5), square))
_assert(not point_in_polygon((2,2), square))
_assert(point_in_polygon((0,0.5), square))

print("SELF-CHECK OK ✅")

### 4) **Визуализация**

In [ ]:
# TODO